In [68]:
import pandas as pd
import numpy as np
import shapefile
from shapely.geometry import Polygon

In [2]:
sf = shapefile.Reader('../downloads/sf_neighborhoods/geo_export_197f44fb-6cc0-472b-81f7-347deefb57df')
sea = shapefile.Reader('../downloads/seattle_neighborhoods/Neighborhoods')

In [4]:
sf_shapes = sf.shapes()

In [70]:
hood_areas_by_city = {}

def record_to_neighborhood(city, record):
    if city == 'Seattle':
        if record[5] != 'OOO' and record[5][1] != ' ':
            return record[5]
        else:
            return None
    elif city == 'San Francisco':
        return record[2]
    else:
        raise Exception("unsupported city: " + city)
        
def calculate_area_from_coords(points):
    """
    Calculate hood area by converting to radians and multiplying by the square of Earth's radius.
    Assuming a perfect sphere, Earth's radius is 6370 km.
    This isn't the most precise, but especially since we're mostly interested in relative measurements, it'll do.
    Did a quick sanity check using neighborhood measurements via Google searches, and they were close.
    """ 
    points = np.array(points)
    poly = Polygon(np.radians(points))
    return poly.area * 6370**2


# Iterate through shapefiles to extract neighborhood names and areas
for city, polyfile in (('San Francisco', sf), ('Seattle', sea)):
    for record, shape in zip(polyfile.records(), polyfile.shapes()):
        name = record_to_neighborhood(city, record)
        if name:
            if city not in hood_areas_by_city:
                hood_areas_by_city[city] = []
            
            points = [list(reversed(p)) for p in shape.points]
            area = calculate_area_from_coords(points)
            hood_areas_by_city[city].append([name, area])

In [73]:
sf_hood_areas = pd.DataFrame(hood_areas_by_city['San Francisco'])
sf_hood_areas.columns = ['Neighborhood', 'Area (sq km)']
sea_hood_areas = pd.DataFrame(hood_areas_by_city['Seattle'])
sea_hood_areas.columns = ['Neighborhood', 'Area (sq km)']
sea_hood_areas.head()

,Neighborhood,Area (sq km)
0,Loyal Heights,2.933042
1,Adams,3.085047
2,Whittier Heights,1.952875
3,West Woodland,3.053022
4,Phinney Ridge,4.418331


In [57]:
financial_district = hoods_by_city['San Francisco'][17][1]
financial_district.area

0.00011132456220592399

In [45]:
fd_points = financial_district.exterior.coords

In [58]:
fidi = sf_shapes[17].points
fidi = np.array(fidi)
fidi_poly = Polygon(np.radians(fidi))
fidi_poly.area * 6370**2

1.3760195231001835

In [48]:
fd_points.xy

(array('d', [37.7957159318478, 37.79571587521785, 37.79534525744691, 37.794952693174395, 37.79468342176938, 37.79447971814928, 37.79325634992732, 37.79309708320577, 37.792405521440855, 37.79101664894328, 37.78926432244501, 37.78872115229309, 37.78764184853344, 37.78796309163017, 37.78842726155285, 37.78889442951887, 37.78880666340405, 37.78869399202777, 37.788499479753874, 37.78842968302915, 37.79212007191527, 37.79772322245175, 37.798540648672464, 37.79919831797124, 37.79921303976896, 37.79883180617899, 37.79885584078806, 37.79882204757367, 37.79885045132736, 37.79884661085972, 37.79883310483591, 37.79933397921428, 37.79940103709016, 37.7994596521181, 37.79946247143917, 37.799625711074924, 37.800049529854206, 37.800063722876935, 37.800110075620914, 37.800143897245185, 37.800181251223826, 37.80015740961311, 37.8000736529846, 37.79999410905789, 37.799956040986764, 37.79998640252755, 37.79998397321377, 37.79999816659603, 37.79944222953126, 37.79940759284774, 37.799348967163674, 37.799286